In [28]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score as auc, accuracy_score
import numpy as np

In [29]:
n = 500
df = pd.read_csv('../prepare_data/data/base_line.csv')
df.drop('match_id', axis=1, inplace=True)
df.head()

,gameweek_gameweek,gameweek_compSeason_label,teams_team_1_name,teams_team_2_name,ground_name,team_1_hue
0,21,2023,Manchester United,Tottenham Hotspur,Old Trafford,2
1,21,2023,Everton,Aston Villa,Goodison Park,2
2,21,2023,Newcastle United,Manchester City,St. James' Park,0
3,21,2023,Chelsea,Fulham,Stamford Bridge,1
4,21,2023,Burnley,Luton Town,Turf Moor,2


In [30]:
train = df[df['gameweek_compSeason_label'] !=2023]
val = df[df['gameweek_compSeason_label'] ==2023]

In [31]:
train

,gameweek_gameweek,gameweek_compSeason_label,teams_team_1_name,teams_team_2_name,ground_name,team_1_hue
203,38,2022,Arsenal,Wolverhampton Wanderers,Emirates Stadium,1
204,38,2022,Aston Villa,Brighton & Hove Albion,Villa Park,1
205,38,2022,Brentford,Manchester City,Gtech Community Stadium,1
206,38,2022,Chelsea,Newcastle United,Stamford Bridge,2
207,38,2022,Crystal Palace,Nottingham Forest,Selhurst Park,2
...,...,...,...,...,...,...
3618,1,2014,Queens Park Rangers,Hull City,Loftus Road,0
3619,1,2014,Stoke City,Aston Villa,bet365 Stadium,0
3620,1,2014,West Bromwich Albion,Sunderland,The Hawthorns,2
3621,1,2014,West Ham United,Tottenham Hotspur,Boleyn Ground,0


In [32]:
y_train = train['team_1_hue']
x_train = train.drop('team_1_hue', axis=1)

y_val = val['team_1_hue']
x_val = val.drop('team_1_hue', axis=1)

In [33]:
model_baseline = CatBoostClassifier()

In [34]:
model_baseline.fit(x_train, y_train, cat_features=list(x_train.columns), verbose=False)

In [35]:
pred = model_baseline.predict(x_val)
pred_proba = model_baseline.predict_proba(x_val)
y_val = y_val.to_numpy().reshape(-1, 1)

In [36]:
pred = np.concatenate((y_val, pred, pred_proba), axis=1)
df_pred = pd.DataFrame(pred, columns=['true', 'pred', '0_prob', '1_prob', '2_prob'])

In [37]:
df_pred['error'] = np.where(df_pred['true']==df_pred['pred'], 0, 1)
df_pred.head()


,true,pred,0_prob,1_prob,2_prob,error
0,2.0,2.0,0.146261,0.350680,0.503059,0
1,2.0,1.0,0.193553,0.412802,0.393646,1
2,0.0,0.0,0.579854,0.142503,0.277643,0
3,1.0,1.0,0.103978,0.695125,0.200896,0
4,2.0,1.0,0.212616,0.505047,0.282336,1


In [38]:
df_pred['error'].sum()

95

In [39]:
accuracy_score(df_pred['true'], df_pred['pred'])

0.5320197044334976